# Data to LED Examples

This notebook will adjust LEDS connected to pins 18-21

# Steps to use a numbered GPIO PIN
1. import GPIO library
2. clear the warnings
3. set GPIO.BCM mode
4. set the pin as output


In [1]:
print ("Current working directory:")
!pwd
print ("\nAvailable files:")
!ls *.csv

Current working directory:
/home/pi/Desktop/data

Available files:
recordings-072921-161554.csv  recordings-073021-114357.csv
recordings-073021-111006.csv  recordings-073021-151701.csv


In [2]:
# Insert the file name
datafile = 'recordings-073021-151701.csv'

In [3]:
# Libraries needed to do the work
import pandas as pd # The Pandas library makes it easier to work with data
import numpy as np # Numpy adds additional numerical methods (math functions)
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

# Read in the csv datafile from above, and save it to the Pandas "dataframe" named sensor_data
# and add the header names to match the data
sensor_data = pd.read_csv(datafile)

# Print the first 5 lines of the dataframe named sensor_data
sensor_data.head()

,date,time,temperature,humidity,pressure,lux,oxidizing,reducing,nh3
0,07/30/21,15:17:01,22.501424,94.338431,695.425250,29.04625,15544.715447,695219.512195,76473.11828
1,07/30/21,15:17:02,32.140563,42.397307,989.865529,12.96480,15544.715447,695219.512195,76473.11828
2,07/30/21,15:17:03,32.141544,42.397305,989.867037,12.96480,15544.715447,704493.827160,76473.11828
3,07/30/21,15:17:04,32.144486,42.397298,989.871561,12.96480,15544.715447,695219.512195,76473.11828
4,07/30/21,15:17:05,32.146121,42.403372,989.869022,12.96480,15544.715447,695219.512195,76473.11828


In [4]:
import RPi.GPIO as GPIO   ## Library that allows the GPIO pins to be used
import time
##Setup the GPIO Pins
for pin in range(18,22):
    GPIO.setwarnings(False)  ## Removes any warnings if the pin was previously used
    GPIO.setmode(GPIO.BCM)   ## Allows the use of the GPIO pin numbers instead of the physical pin numbers
    GPIO.setup(pin,GPIO.OUT)
    GPIO.output(pin,GPIO.LOW) ## Sets GPIO pin 21 output voltage low/off

# Lights adjust based on the Humidity readings

More leds will turn on based on the humidity reading found in the data set

In [6]:
max_humidity = sensor_data["humidity"].max()
print("The maximum humidity in the data is: ", max_humidity)

The maximum humidity in the data is:  94.33843134700204


In [8]:
# Using NumPY Interpolation the below interpolates the sensors data into a given range which in
# this case are the number of LEDS connected
# np.interp(sensorvalue, [min of all sensorvalues, max of all sensorvalues] , [first led pin, last led pin])
int(np.interp(max_humidity, [sensor_data["humidity"].min(),sensor_data["humidity"].max()], [18,21]))

21

In [10]:
## Cycle through all the values in the data set under Humidity
for sensorvalue in sensor_data["humidity"]:
    #Interpolate the current value
    pin = int(np.interp(sensorvalue, [sensor_data["humidity"].min(),sensor_data["humidity"].max()], [18,21]))
    #Turn on the number of pins up to the interpolated value
    for pins in range(18, 22):
        GPIO.output(pins,GPIO.HIGH)
    time.sleep(2)
    #Turn all the pins off before checking the next value
    for pin_off in range(18,22):
        GPIO.output(pins,GPIO.LOW)
    time.sleep(1)
# Turn all pins off
for pin_off in range(18,22):
        GPIO.output(pin_off,GPIO.LOW)

# Stop Light Colors from Data

Connect:
* Red LED to pin 18
* Yellow LED to pin 19
* Green LED to pin 20

In [12]:
#Using Analog channel 0 (A0) for this example
# np.interp(sensorvalue, [min of all sensorvalues, max of all sensorvalues] , [first led pin, last led pin])
max_a0 = sensor_data["lux"].max()
print("The maximum value for lux in the data is: ", max_a0)
print(int(np.interp(max_a0, [sensor_data["lux"].min(),sensor_data["lux"].max()], [18,20])))
min_a0 = sensor_data["lux"].min()
print("The min value for lux in the data is: ", min_a0)
print(int(np.interp(min_a0, [sensor_data["lux"].min(),sensor_data["lux"].max()], [18,20])))
mean_a0 = sensor_data["lux"].mean()
print("The mean value for lux in the data is: ", mean_a0)
print(int(np.interp(mean_a0, [sensor_data["lux"].min(),sensor_data["lux"].max()], [18,20])))

The maximum value for lux in the data is:  29.04625
20
The min value for lux in the data is:  12.9648
18
The mean value for lux in the data is:  15.310590833333334
18


#### Note: Analog sensors generally have higher readings when nothing is detected and lower reading when something is detected. 

In [15]:
# An easily changeable value to pick which sensor to use
sensor_value_name = "nh3"
pins_to_use = [18,20]
# Cycle through all the values in the data set under Humidity
for sensorvalue in sensor_data[sensor_value_name]:
    #Interpolate the current value
    pin = int(np.interp(sensorvalue, [sensor_data[sensor_value_name].min(),sensor_data[sensor_value_name].max()], pins_to_use))
    GPIO.output(pin,GPIO.HIGH)
    time.sleep(2)
    #Turn all the pins off before checking the next value
    GPIO.output(pin,GPIO.LOW)
    time.sleep(1)


# Turn all LEDS (pins) Off

In [8]:
for pin in range (18,22):
     GPIO.output(pin,GPIO.LOW)